## **Riot API Tutorial (2025-02-25)**

*This sandbox follows the youtube tutorial series by Beora ([Youtube Link](https://www.youtube.com/watch?v=A5c0yfEiYjQ&list=PLnnM9dgSdUvIe0MRJcjbzBaJWFHNJaOh1)). It is meant to give a basic Introduction to the Tools Necessary to Get Started working on Projects utilising Riot's API.*

### **1. Establishing Credentials**

In [27]:
from dotenv import load_dotenv
import os

load_dotenv()

print(os.environ.get('riot_api_key'))

RGAPI-e6166e33-d300-4f07-a8b9-4d47f978a885


In [28]:
import pygsheets

service_account = pygsheets.authorize(service_account_file='JSONs\\spreadsheet-automator-452009-1b2fd6310821.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1F38cGIAQtiPwX0CjVb742iqwnnMW8Lh2Exy-SRP_BrM/edit?usp=sharing')

In [29]:
test = sheet.worksheet_by_title('Test')

In [30]:
test_df = test.get_as_df()

In [31]:
test_df['random_numbers'] = 1

In [32]:
test.set_dataframe(test_df, 'A1', copy_head=True)

### **2. Interacting with the API**

In [33]:
from dotenv import load_dotenv
load_dotenv()

import os
import requests

gameName = 'LordVoItage'
tagLine = 'EUW'
api_key = os.environ.get('riot_api_key')

def get_puuid(gameName=None, tagLine=None, api_key=None):
    link = f'https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}%20/{tagLine}?api_key={api_key}'

    response = requests.get(link)
    response.json()

    return response.json()['puuid']

def get_name_and_tag(puuid=None, api_key=None):
    link = f'https://europe.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)
    response.json()

    gameName = response.json()['gameName']
    tagLine = response.json()['tagLine']

    return f'{gameName}#{tagLine}'

In [34]:
get_puuid(gameName, tagLine, api_key)

'sYgaAEq5FqgCqn7iIse8zzNHhmQ8CbQP4YVR-Lyg5ZQT_UvjRriqWzHTzHbG2pO_k74mXuHuQw5rJw'

In [35]:
get_name_and_tag(puuid=get_puuid(gameName, tagLine, api_key), api_key=api_key)

'LordVoItage#EUW'

### **3. Retrieving the SoloQ Ladder**

In [36]:
response = requests.get(f'https://euw1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}')

puuid = response.json()['entries'][0]['puuid']

summoner_name = get_name_and_tag(puuid, api_key)

summoner_name

'G2 SkewMond#3327'

In [37]:
import pandas as pd

def get_ladder(top=None, api_key=None):
    root = 'https://euw1.api.riotgames.com/'
    chall = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gmaster = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    master = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

    chall_response = requests.get(root + chall + '?api_key=' + api_key)
    gmaster_response = requests.get(root + gmaster + '?api_key=' + api_key)
    master_response = requests.get(root + master + '?api_key=' + api_key)

    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    gmaster_df = pd.DataFrame(gmaster_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    master_df = pd.DataFrame(master_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    ladder = pd.concat([chall_df, gmaster_df, master_df], keys=['Challenger', 'Grandmaster', 'Master'])[:top].reset_index(drop=True)

    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index': 'rank'})
    ladder['rank'] += 1

    return ladder

In [38]:
ladder = get_ladder(top=10, api_key=api_key)

In [39]:
ladder['summoner_name'] = ladder['puuid'].apply(lambda x: get_name_and_tag(x, api_key))

In [40]:
ladder

,rank,summonerId,puuid,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak,summoner_name
0,1,aWgR5OW8ruiR-BZy2CmyE01u6iWSsiZrTO_JJ5tVK01_oriX,vGfPE8LWdIKRLLC0huTNYNKwSJ4dyesDd4J5tVy8LSUNUl...,2131,187,125,True,False,False,False,G2 SkewMond#3327
1,2,fpCdtOdci7tnvJDPdNC1U6KDRs_lq64Op6mqeUyKVLlzrd...,ZhxdGQCRKO7k1FUjbQfGO2Ln-Dxl4n9oA9txYN2rqhoyis...,1875,234,179,True,False,False,False,J1HUIV#000
2,3,8G_VDEDjVDXP9mXgESWI_9bILOaUnPHsrhB6PJnwbqse97o,Az2f_S0MvTr-7m97Wsiyj_i0ZGKSPrG9hIXJv5tSYYG4sg...,1710,129,80,True,False,False,True,Odysseus#131
3,4,0fPtqO_Ptj8EiqYaGlbv5Z6wZ6D_7oYqnMJmeTlNF7FIeQw,QtOWvEaM-3H3cFdQLbH3qILSwZDBbcQOaT4-aTSNEZWjVV...,1700,155,112,True,False,False,False,G2 Labrov#8085
4,5,iTeb8_FNg2J4aDakvghhJUbm-002iasywLms0iUF_4PM7n...,HAJ71n_egQn7Dkfw9y6CXPQ9N4Cnrdxm1j5RIpbIjaieL2...,1679,240,160,True,False,False,False,Ivory#EfEf
5,6,ujQ7AWE6GNKTVHk1wkjX4agW3qBdd3t82CXBWBruAB5jcS9k,wQ3sHQG8e56XQa_zEUsnMOVihPr4v2pnsb2HkZjuFfWleY...,1593,130,86,True,False,False,False,TwTv Phantasm#103
6,7,9BSiPqVSq27Var4YFNZaqabQsXODdH2XSi02ZzNCBMhGo5Y,Fn2pIdf1zFAtFPBUicQdoztmbEDoUxV1W8pnc6Sc_LbeCl...,1582,155,115,True,False,False,False,G2 Hans Sama#12838
7,8,SMdG7MT4Pc3ruok6Z4IPA9ZhwlBAot8Le5zxZqPKsqNFHgjE,JysLF52n9WgPlvK4ecAuCUId-ILYS0U-kd4obqmSB0YL9t...,1579,199,164,True,False,False,False,Tοαst#EUW
8,9,KHBIZVoYC50_YO0oSJ5rXtBErZL8kIozDAXJIiYlWcJkdCGt,7PHuKgmARX_YeXzyRninl-SriUyX9E4PKa44ckoLdv630B...,1557,113,79,True,False,False,False,KC NEXT ADKING#EUW
9,10,5mPvdoGCJoaSNr3Sc8LCn_srhWnYC9YDU3opWVqfsSO_kGfs,TL0sOMds7kvWSKxxfCmiZRphzERZCv2CEwfQ42r_IiRhrv...,1556,213,141,True,False,False,False,Vetheø#4321


### **4. Obtaining Match Histories**

In [41]:
def get_match_history(puuid=None, api_key=None, start=0, count=20):

    root = 'https://europe.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?type=ranked&start={start}&count={count}'

    response = requests.get(root + endpoint + query_params + '&api_key=' + api_key)

    return response.json()

def get_match_data_from_id(matchId=None, api_key=None):

    root = 'https://europe.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/{matchId}'

    response = requests.get(root + endpoint + '?api_key=' + api_key)

    return response.json()


In [42]:
puuid = get_puuid(gameName, tagLine, api_key)
matchId = get_match_history(puuid, api_key, start=0, count=100)[0]

get_match_data_from_id(matchId, api_key)

{'metadata': {'dataVersion': '2',
  'matchId': 'EUW1_7317675815',
  'participants': ['9yECzY_qZRykY05W1qz37r4loWBQjA0gF3rNuEuqqHmtloyDeg2I4LpTm5sT9L2uMp-oIjB8_BfNdg',
   'U7aHYNfy8KKaGl46wvJA24ag-TYUOcGin_ARtlniAnxpjdpK1NZvdKUcdsEb1CYWrKBHXFT4zNQXPQ',
   '6hBf0MBsJkWo4vTDXJ6WM2u024Ww04JSMDsD06xf9zR4xiT7O7Pg59ZT-3O644mM4-Fl_a2NaCHYhQ',
   'NEzHGIamYgeOxvlJSxCjhYjK89xHCvXlueQyJVVQQwFL9EA5XN0Tjqb6CSNZxiwezhrGW4KBUAGSOw',
   '7rnquCJUtUU17bYV9tweVpi1HzfvdICy0csPEMcTqgwjwPB2ra8MaTBU5Ci4Goch51zHROkLTCc_uQ',
   'H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoMECtc68UQKDURkNLNvDo68UU9pEUkMHEog',
   'sYgaAEq5FqgCqn7iIse8zzNHhmQ8CbQP4YVR-Lyg5ZQT_UvjRriqWzHTzHbG2pO_k74mXuHuQw5rJw',
   '7J7ikEoyDFQMEQeNojtzE_LIjeW2qJZXirGZuZn_olvip6yaRxyeHUcuG4YJxt22UZMqTufcbf0hMg',
   'On8aKn4RxhweTnG6bsL_WiJuXoJRpLdEToYTT0L7_3xqz-uOPKgt-f7SaBKhLvm12p_y4YyOgU9inw',
   'IvzgjokXfwX4uQv52XMWmIAKYdkYF38QkoNi7bdkSbfNqxFyAZVw4lQIz6yk6Hj2ZxcCkW_lYXMnzw']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 174

### **5. Processing SoloQ Matches**

In [43]:
def process_match_json(match_json, puuid=None):

    ## Architecture
    metadata = match_json['metadata']
    info = match_json['info']
    players = info['participants']
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(puuid)]
    perks = player['perks']
    stats = perks['statPerks']
    styles = perks['styles']

    primary = styles[0]
    secondary = styles[1]

    match_id = metadata['matchId']

    ## Data
    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_timestamp = info['gameEndTimestamp']
    patch = info['gameVersion']

    riot_id = player['riotIdGameName']
    riot_tag = player['riotIdTagline']
    summoner_id = player['summonerId']
    summoner_name = player['summonerName']

    win = player['win']

    champ_id = player['championId']
    champ_transform = player['championTransform']
    champ_level = player['champLevel']

    gold_earned = player['goldEarned']
    neutral_minions_killed = player['neutralMinionsKilled']
    total_minions_killed = player['totalMinionsKilled']

    kills = player['kills']
    deaths = player['deaths']
    assists = player['assists']
    first_blood = player['firstBloodKill']

    total_damage_dealt = player['totalDamageDealt']
    total_damage_shielded = player['totalDamageShieldedOnTeammates']
    total_damage_taken = player['totalDamageTaken']
    total_time_cc_dealt = player['totalTimeCCDealt']

    early_surrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']

    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']

    summoner_1_id = player['summoner1Id']
    summoner_2_id = player['summoner2Id']

    wards_placed = player['wardsPlaced']
    wards_killed = player['wardsKilled']
    vision_score = player['visionScore']

    objectives_stolen = player['objectivesStolen']
    objectives_stolen_assists = player['objectivesStolenAssists']

    defense = stats['defense']
    flex = stats['flex']
    offense = stats['offense']

    primary_style = primary['style']
    secondary_style = primary['style']

    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1  = primary['selections'][1]['perk']
    primary_perk_2  = primary['selections'][2]['perk']
    primary_perk_3  = primary['selections'][3]['perk']

    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']

    #for team in teams:
    #    if team['teamId'] == player['teamId']:
    #        objs = team['objectives']
    #        baron = objs['baron']
    #        dragon = objs['dragon']
    #        grubs = objs['horde']
    #        rift_herald = objs['riftHerald']
    #        tower = objs['tower']
    #        inhibitor = objs['inhibitor']
    

    match_df = pd.DataFrame({
        'match_id': [match_id],
        'participants': [participants],
        'game_creation': [game_creation],
        'game_duration': [game_duration],
        'game_start_timestamp': [game_start_timestamp],
        'game_end_timestamp': [game_end_timestamp],
        'patch': [patch],
        'riot_id': [riot_id],
        'riot_tag': [riot_tag],
        'summoner_id': [summoner_id],
        'summoner_name': [summoner_name],
        'win': [win],
        'champ_id': [champ_id],
        'champ_transform': [champ_transform],
        'champ_level': [champ_level],
        'gold_earned': [gold_earned],
        'neutral_minions_killed': [neutral_minions_killed],
        'total_minions_killed': [total_minions_killed],
        'kills': [kills],
        'deaths': [deaths],
        'assists': [assists],
        'first_blood': [first_blood],
        'total_damage_dealt': [total_damage_dealt],
        'total_damage_shielded': [total_damage_shielded],
        'total_damage_taken': [total_damage_taken],
        'total_time_cc_dealt': [total_time_cc_dealt],
        'early_surrender': [early_surrender],
        'surrender': [surrender],
        'item0': [item0],
        'item1': [item1],
        'item2': [item2],
        'item3': [item3],
        'item4': [item4],
        'item5': [item5],
        'item6': [item6],
        'summoner_1_id': [summoner_1_id],
        'summoner_2_id': [summoner_2_id],
        'wards_placed': [wards_placed],
        'wards_killed': [wards_killed],
        'vision_score': [vision_score],
        'objectives_stolen': [objectives_stolen],
        'objectives_stolen_assists': [objectives_stolen_assists],
        'defense': [defense],
        'flex': [flex],
        'offense': [offense],
        'primary_style': [primary_style],
        'secondary_style': [secondary_style],
        'primary_keystone': [primary_keystone],
        'primary_perk_1': [primary_perk_1],
        'primary_perk_2': [primary_perk_2],
        'primary_perk_3': [primary_perk_3],
        'secondary_perk_1': [secondary_perk_1],
        'secondary_perk_2': [secondary_perk_2],
        })
    return match_df



In [44]:
gameName = 'LordVoItage'
tagLine = 'EUW'
api_key = os.environ.get('riot_api_key')

puuid = get_puuid(gameName, tagLine, api_key)
matchId = get_match_history(puuid, api_key, start=0, count=100)[2]

game = get_match_data_from_id(matchId, api_key)

process_match_json(game, puuid)

,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,riot_id,riot_tag,summoner_id,...,flex,offense,primary_style,secondary_style,primary_keystone,primary_perk_1,primary_perk_2,primary_perk_3,secondary_perk_1,secondary_perk_2
0,EUW1_7315801214,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1740258651379,1830,1740258775404,1740260605495,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,...,5008,5008,8100,8100,8128,8143,8140,8135,9111,8014


In [45]:
match_ids = get_match_history(puuid, api_key, start=0, count=5)

df = pd.DataFrame()
for match_id in match_ids:
    match = get_match_data_from_id(match_id, api_key)
    match_df = process_match_json(match, puuid)
    
    df = pd.concat([df, match_df])

In [46]:
df

,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,riot_id,riot_tag,summoner_id,...,flex,offense,primary_style,secondary_style,primary_keystone,primary_perk_1,primary_perk_2,primary_perk_3,secondary_perk_1,secondary_perk_2
0,EUW1_7317675815,[9yECzY_qZRykY05W1qz37r4loWBQjA0gF3rNuEuqqHmtl...,1740410317834,1703,1740410350088,1740412053643,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,...,5008,5008,8100,8100,8128,8143,8140,8135,9111,8014
0,EUW1_7316614390,[ane_rqP8NlA6dPk3xjdamKA91ZALfRyVekIxSmWI6YCbV...,1740325731450,1597,1740325754586,1740327352070,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,...,5008,5008,8100,8100,8128,8143,8140,8135,9111,8014
0,EUW1_7315801214,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1740258651379,1830,1740258775404,1740260605495,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,...,5008,5008,8100,8100,8128,8143,8140,8135,9111,8014
0,EUW1_7315747405,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1740256517587,1719,1740256568697,1740258288305,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,...,5008,5008,8100,8100,8128,8143,8140,8135,9111,8014
0,EUW1_7311182902,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1739908043173,2410,1739908071002,1739910481503,15.3.656.4086,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,...,5008,5008,8100,8100,8128,8143,8140,8135,9111,8014


### **6. Utlising Community Dragon**

In [47]:
perk = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json'
perk_styles = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json'

perk_json = requests.get(perk).json()
perk_styles_json = requests.get(perk_styles).json()

In [48]:
def json_extract(obj, key):

    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)

        return arr
    
    values = extract(obj, arr, key)
    return values

In [49]:
perk_ids = json_extract(perk_json, 'id')
perk_names = json_extract(perk_json, 'name')

# perk_dict = dict(map(lambda x, y: (x, y), perk_ids, perk_names))
perk_dict = dict(zip(perk_ids, perk_names))


perk_styles_ids = json_extract(perk_styles_json, 'id')
perk_styles_names = json_extract(perk_styles_json, 'name')

perk_styles_dict = dict(map(lambda x, y: (int(x), y), perk_styles_ids, perk_styles_names))
# perk_styles_dict = dict(zip(perk_styles_ids, perk_styles_names))

In [50]:
pd.options.display.max_columns = 100

df = df.replace(perk_dict).replace(perk_styles_dict)

df

,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,riot_id,riot_tag,summoner_id,summoner_name,win,champ_id,champ_transform,champ_level,gold_earned,neutral_minions_killed,total_minions_killed,kills,deaths,assists,first_blood,total_damage_dealt,total_damage_shielded,total_damage_taken,total_time_cc_dealt,early_surrender,surrender,item0,item1,item2,item3,item4,item5,item6,summoner_1_id,summoner_2_id,wards_placed,wards_killed,vision_score,objectives_stolen,objectives_stolen_assists,defense,flex,offense,primary_style,secondary_style,primary_keystone,primary_perk_1,primary_perk_2,primary_perk_3,secondary_perk_1,secondary_perk_2
0,EUW1_7317675815,[9yECzY_qZRykY05W1qz37r4loWBQjA0gF3rNuEuqqHmtl...,1740410317834,1703,1740410350088,1740412053643,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,16,12783,181,6,7,2,10,True,254406,0,30676,231,False,False,6701,3171,3142,3814,1036,1036,3364,4,11,2,0,15,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
0,EUW1_7316614390,[ane_rqP8NlA6dPk3xjdamKA91ZALfRyVekIxSmWI6YCbV...,1740325731450,1597,1740325754586,1740327352070,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,16,12170,179,16,6,0,13,False,256447,0,24319,215,False,False,6701,3134,3142,2022,1001,1036,3364,4,11,3,1,25,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
0,EUW1_7315801214,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1740258651379,1830,1740258775404,1740260605495,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,17,13266,180,23,9,2,7,False,267159,0,36601,287,False,False,6701,3814,3142,3111,1036,0,3364,4,11,2,1,13,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
0,EUW1_7315747405,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1740256517587,1719,1740256568697,1740258288305,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,17,13779,182,18,13,2,11,False,268289,0,36992,150,False,False,6701,3814,3142,3158,6694,0,3364,4,11,1,4,20,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
0,EUW1_7311182902,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1739908043173,2410,1739908071002,1739910481503,15.3.656.4086,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,False,121,0,18,16008,180,20,13,7,7,False,318618,0,47874,348,False,False,6701,3171,3142,3814,6694,3134,3364,4,11,1,2,22,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace


### **7. Storing Data in SQL**

In [51]:
import os
from dotenv import load_dotenv
load_dotenv()

from pangres import upsert
from sqlalchemy import create_engine, text

db_host=os.environ.get('db_host')
db_port=os.environ.get('db_port')
db_name=os.environ.get('db_name')
db_username=os.environ.get('db_username')
db_password=os.environ.get('db_password')


def create_db_connection_url(db_host=None, db_port=None, db_name=None, db_username=None, db_password=None):
    connection_url = f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'
    return connection_url

conn = create_db_connection_url(db_host, db_port, db_name, db_username, db_password)

db_engine = create_engine(conn, pool_recycle=3600)

connection = db_engine.connect()

In [ ]:
df['uuid'] = df['match_id'] + '_' + df['summoner_id']
df.set_index('uuid', inplace=True)

df

,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,riot_id,riot_tag,summoner_id,summoner_name,win,champ_id,champ_transform,champ_level,gold_earned,neutral_minions_killed,total_minions_killed,kills,deaths,assists,first_blood,total_damage_dealt,total_damage_shielded,total_damage_taken,total_time_cc_dealt,early_surrender,surrender,item0,item1,item2,item3,item4,item5,item6,summoner_1_id,summoner_2_id,wards_placed,wards_killed,vision_score,objectives_stolen,objectives_stolen_assists,defense,flex,offense,primary_style,secondary_style,primary_keystone,primary_perk_1,primary_perk_2,primary_perk_3,secondary_perk_1,secondary_perk_2
uuid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
EUW1_7317675815_N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,EUW1_7317675815,[9yECzY_qZRykY05W1qz37r4loWBQjA0gF3rNuEuqqHmtl...,1740410317834,1703,1740410350088,1740412053643,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,16,12783,181,6,7,2,10,True,254406,0,30676,231,False,False,6701,3171,3142,3814,1036,1036,3364,4,11,2,0,15,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
EUW1_7316614390_N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,EUW1_7316614390,[ane_rqP8NlA6dPk3xjdamKA91ZALfRyVekIxSmWI6YCbV...,1740325731450,1597,1740325754586,1740327352070,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,16,12170,179,16,6,0,13,False,256447,0,24319,215,False,False,6701,3134,3142,2022,1001,1036,3364,4,11,3,1,25,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
EUW1_7315801214_N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,EUW1_7315801214,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1740258651379,1830,1740258775404,1740260605495,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,17,13266,180,23,9,2,7,False,267159,0,36601,287,False,False,6701,3814,3142,3111,1036,0,3364,4,11,2,1,13,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
EUW1_7315747405_N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,EUW1_7315747405,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1740256517587,1719,1740256568697,1740258288305,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,17,13779,182,18,13,2,11,False,268289,0,36992,150,False,False,6701,3814,3142,3158,6694,0,3364,4,11,1,4,20,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
EUW1_7311182902_N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,EUW1_7311182902,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1739908043173,2410,1739908071002,1739910481503,15.3.656.4086,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,False,121,0,18,16008,180,20,13,7,7,False,318618,0,47874,348,False,False,6701,3171,3142,3814,6694,3134,3364,4,11,1,2,22,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace


In [57]:
upsert(con=connection, df=df, schema='soloq', table_name='regional_player_matches', create_table=True, create_schema=True, if_row_exists='update')

In [58]:
connection.commit()

In [62]:
def get_soloq_games(side=None, db_engine=None):
    with db_engine.connect() as connection:
        df = pd.read_sql(text(f"""SELECT 
                                    * 
                                  FROM 
                                    soloq.regional_player_matches
                                  WHERE
                                    side = '{side}"""), connection)
    return df

In [60]:
df

,uuid,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,riot_id,riot_tag,summoner_id,summoner_name,win,champ_id,champ_transform,champ_level,gold_earned,neutral_minions_killed,total_minions_killed,kills,deaths,assists,first_blood,total_damage_dealt,total_damage_shielded,total_damage_taken,total_time_cc_dealt,early_surrender,surrender,item0,item1,item2,item3,item4,item5,item6,summoner_1_id,summoner_2_id,wards_placed,wards_killed,vision_score,objectives_stolen,objectives_stolen_assists,defense,flex,offense,primary_style,secondary_style,primary_keystone,primary_perk_1,primary_perk_2,primary_perk_3,secondary_perk_1,secondary_perk_2
0,EUW1_7317675815_N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsS...,EUW1_7317675815,[9yECzY_qZRykY05W1qz37r4loWBQjA0gF3rNuEuqqHmtl...,1740410317834,1703,1740410350088,1740412053643,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,16,12783,181,6,7,2,10,True,254406,0,30676,231,False,False,6701,3171,3142,3814,1036,1036,3364,4,11,2,0,15,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
1,EUW1_7316614390_N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsS...,EUW1_7316614390,[ane_rqP8NlA6dPk3xjdamKA91ZALfRyVekIxSmWI6YCbV...,1740325731450,1597,1740325754586,1740327352070,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,16,12170,179,16,6,0,13,False,256447,0,24319,215,False,False,6701,3134,3142,2022,1001,1036,3364,4,11,3,1,25,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
2,EUW1_7315801214_N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsS...,EUW1_7315801214,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1740258651379,1830,1740258775404,1740260605495,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,17,13266,180,23,9,2,7,False,267159,0,36601,287,False,False,6701,3814,3142,3111,1036,0,3364,4,11,2,1,13,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
3,EUW1_7315747405_N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsS...,EUW1_7315747405,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1740256517587,1719,1740256568697,1740258288305,15.4.657.5386,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,True,121,0,17,13779,182,18,13,2,11,False,268289,0,36992,150,False,False,6701,3814,3142,3158,6694,0,3364,4,11,1,4,20,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
4,EUW1_7311182902_N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsS...,EUW1_7311182902,[H_JK6bXTiLE3NiMXecRMaOIE5raLhzWkJ_manTPFxHhoM...,1739908043173,2410,1739908071002,1739910481503,15.3.656.4086,LordVoItage,EUW,N7UjrBLG_BZC-wQf2Nw3QI-ACcUDsSb69YfXsmT48LiztMg,,False,121,0,18,16008,180,20,13,7,7,False,318618,0,47874,348,False,False,6701,3171,3142,3814,6694,3134,3364,4,11,1,2,22,0,0,Health Scaling,Adaptive Force,Adaptive Force,Precision,Precision,Dark Harvest,Sudden Impact,Grisly Mementos,Treasure Hunter,Triumph,Coup de Grace
